In [1]:
test_set_path = "../dataset/test-00000-of-00001.parquet"

In [2]:
import pandas as pd
test_df = pd.read_parquet(test_set_path)

In [3]:
from prompt.prompt import get_sentiment_classifier_prompt
from llm_service.llm_caller import LLMCaller
from config import LLMConfig, load_yaml_config
from utils.load_label_maps import load_label_maps

import time
from tqdm import tqdm

In [4]:
LABEL_MAPS = load_label_maps("../dataset/label_maps.yaml")

In [5]:
load_yaml_config("../config/config.yaml")

In [6]:
llm_config = LLMConfig()
llm_service = LLMCaller(llm_config)

/root/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#total_row = len(test_df)

import os

total_row = len(test_df)
ans_df = pd.DataFrame(columns=["texts", "category", "process_time"])

RESULTS_DIR = "../results/"
results_path = os.path.join(RESULTS_DIR, f"{llm_service.llm_config.MODEL}_results.csv")

if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)

for row in tqdm(range(total_row)):
    sentence = test_df.loc[row, "texts"]
    sentiment_classifier_prompt = get_sentiment_classifier_prompt(message=sentence)
    start_process = time.time()
    ans = llm_service.call(sentiment_classifier_prompt)
    process_time = time.time() - start_process
    try:
        category = LABEL_MAPS[ans]
    except KeyError:
        category = ans

    ans_df.loc[row, "texts"] = sentence
    ans_df.loc[row, "category"] = category
    ans_df.loc[row, "process_time"] = process_time

    ans_df.to_csv(results_path, index=False)

  0%|                                                                                                                              | 0/2671 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1721477062.964215    1789 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1721477062.975811    1789 check_gcp_environment.cc:61] BIOS data file does not exist or cannot be opened.
  4%|█████                                                                                                             | 120/2671 [06:32<1:12:59,  1.72s/it]2024-07-20 12:10:55,657 [ERROR]  gemini-pro request error: 429 Resource has been exhausted (e.g. check quota)., waiting for connecting to gemini-pro again


In [ ]:
def evaluate(test_df: pd.DataFrame, ans_df: pd.DataFrame) -> float:
    correct = 0

    if len(test_df) != len(ans_df):
        raise ValueError("2 dataframes not same length.")

    total_row = len(test_df)
    for row in range(total_row):
        actual = test_df.loc[row, "category"]
        predict = ans_df.loc[row, "category"]
        if actual == predict:
            correct += 1
    return correct / total_row

In [ ]:
acc = evaluate(test_df, ans_df)

In [ ]:
acc